In [1]:
import scanpy as sc
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearColorMapper,ColorBar, LinearAxis, Range1d,LinearScale
from bokeh.models.transforms import CustomJSTransform
from bokeh.transform import factor_cmap, transform
import sys
sys.path.append('../beehive/')
os.environ['BEEHIVE_BASEDIR'] = '/Users/raghidbsat/Extras_No_Icloud_save/VIB/bds_lab/beehive'
import os
import numpy as np
import beehive
from beehive import config, util, expset
from bokeh.palettes import Category20
import polars as pl
from bokeh.util.hex import cartesian_to_axial
from bokeh.transform import jitter


In [2]:
output_notebook()

Loading BokehJS ...

In [98]:

datadir = util.get_datadir("h5ad")
dsid ='h.man2m.1.sct'
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.obs.prq", ["oAbvsScr"])
# rv.to_pandas()

In [282]:
gene = "APOE"
meta = "cell.state"
meta2 = "NLGFapoeKOvsNLGF"
genedata = expset.get_gene(dsid, gene)
metadata = expset.get_meta(dsid, meta)
metadata2 = expset.get_meta(dsid,meta2,raw=True)
mouseids = expset.get_meta(dsid,"mouse.id")


rv_combined = (
        pl.concat([genedata, metadata,metadata2,mouseids], how="horizontal")
        .groupby([meta,meta2])
        .agg(
            [
                pl.count(),
                pl.mean(gene).alias("mean"),
                pl.std(gene).alias("std"),
                pl.median(gene).alias("median"),
                pl.quantile(gene, 0.25).alias("q25"),
                pl.quantile(gene, 0.75).alias("q75"),
                pl.quantile(gene, 0.01).alias("q01"),
                pl.quantile(gene, 0.99).alias("q99"),
                pl.min(gene).alias("min"),
                pl.max(gene).alias("max"),
            ]
        )
    )


rv_mouseid = (
        pl.concat([genedata, metadata,metadata2,mouseids], how="horizontal")
        .groupby([meta,meta2,"mouse.id"])
        .agg(
            [
                pl.count().alias("count_mouseid"),
                pl.mean(gene).alias("mean_mouseid"),
            ]
        )
    )

rv_combined = rv_combined.to_pandas()
rv_mouseid = rv_mouseid.to_pandas()


rv_combined['perc'] = 100 * rv_combined['count'] / rv_combined['count'].sum()
rv_combined['_segment_top'] = rv_combined['q99']
rv_combined['_bar_top'] = rv_combined['q75']
rv_combined['_bar_median'] = rv_combined['median']
rv_combined['_bar_bottom'] = rv_combined['q25']
rv_combined['_segment_bottom'] = rv_combined['q01']


x = [ (genotype, cellstate) for genotype in rv_combined["NLGFapoeKOvsNLGF"].tolist() for cellstate in rv_combined["cell.state"].tolist() ]

rv_combined["x"] = list(set(x))
rv_combined["x"] = sorted(rv_combined["x"],key=lambda tup: tup[0])

rv_mouseid["x"] = rv_mouseid[["NLGFapoeKOvsNLGF","cell.state"]].apply(tuple, axis=1)
rv_mouseid["x"] = sorted(rv_mouseid["x"],key=lambda tup: tup[0])


final_rv = pd.merge(rv_combined,rv_mouseid,on="x")

final_rv["x"] = sorted(final_rv["x"],key=lambda tup: tup[0])

source2 = ColumnDataSource(final_rv)

plot2 = figure(x_range = [], width  = 1000, height = 500)
plot2.x_range.factors = sorted(list(set(x)),key=lambda tup: tup[0])


vbar=plot2.vbar(x="x", top='_bar_top',
                bottom='_bar_bottom', source = source2, width=0.85, name="barplot",
                fill_color=config.colors.color6,
                line_color="black"),
jitter_points = plot2.scatter(x=jitter('x', width=0.4, range=plot2.x_range), y='mean_mouseid', size=5, alpha=0.4, source=source2)


# seg_v_up=plot2.segment(source=source2, x0='x', x1='x',
#                         y0='_bar_top', y1='_segment_top',
#                         line_color='black'),
# seg_h_up=plot2.rect(source=source2, x='x', height=0.001,
#                     y='_segment_top', width=0.4, line_color='black'),
# seg_v_dw=plot2.segment(source=source2, x0='x', x1='x',
#                         y0='_segment_bottom', y1='_bar_bottom',
#                         line_color='black'),
# seg_h_dw=plot2.rect(source=source2, x='x', height=0.001,
#                     y='_segment_bottom', width=0.4, line_color='black'),
# seg_h_med=plot2.rect(source=source2, x='x', height=0.001,
#                     y='_bar_median', width=0.85, line_width=2,
#                     line_color='black'),

plot2.xaxis.major_label_orientation = 3.14/2
        
# show(plot2)


In [6]:
gene = "APOE"
meta = "cell.state"
meta2 = "NLGFapoeKOvsNLGF"
meta3 = "mouse.id"
dsid ='h.man2m.1.sct'
final_rv,factors = expset.get_gene_meta_three_facets(dsid,gene,meta,meta2,meta3)
source = ColumnDataSource(final_rv)


###plot###
plot = figure(x_range = [], width  = 1000, height = 500)
plot.x_range.factors = sorted(list(set(factors)),key=lambda tup: tup[0])


vbar=plot.vbar(x="x", top='_bar_top',
                bottom='_bar_bottom', source = source, width=0.85, name="barplot",
                fill_color=config.colors.color6,
                line_color="black"),
jitter_points = plot.scatter(x=jitter('x', width=0.4, range=plot.x_range), y=f'mean_{meta3}', size=5, alpha=0.4, source=source)
plot.xaxis.major_label_orientation = 3.14/2
show(plot)

In [179]:
gene = "APOE"
meta = "cell.state"

genedata = expset.get_gene(dsid, gene)
metadata = expset.get_meta(dsid, meta)
mouseids = expset.get_meta(dsid,"mouse.id")

rv = (
        pl.concat([genedata, metadata,mouseids], how="horizontal")
        .groupby(meta)
        .agg(
            [
                pl.count(),
                pl.mean(gene).alias("mean"),
                pl.std(gene).alias("std"),
                pl.median(gene).alias("median"),
                pl.quantile(gene, 0.25).alias("q25"),
                pl.quantile(gene, 0.75).alias("q75"),
                pl.quantile(gene, 0.01).alias("q01"),
                pl.quantile(gene, 0.99).alias("q99"),
                pl.min(gene).alias("min"),
                pl.max(gene).alias("max"),
            ]
        )
    )

rv_mouseid = (
        pl.concat([genedata, metadata,mouseids], how="horizontal")
        .groupby([meta,"mouse.id"])
        .agg(
            [
                pl.count().alias("count_mouseid"),
                pl.mean(gene).alias("mean_mouseid"),
            ]
        )
    )





# rv = rv.to_pandas()
rv = rv.to_pandas()
rv = rv.rename(columns={meta: "cat_value"})

rv_mouseid= rv_mouseid.to_pandas()
rv_mouseid = rv_mouseid.rename(columns={meta: "cat_value"})

# Does the YAML have sort_order fields? If so use these
# if False:  # if there are order fields:
#     # TODO: Raghid
#     pass
# else:
# Otherwise - do the following:
try:
    # attempt a numerical sort on the cat_value
    # try to prevent alphanumerically sorted numeric
    # values 1 10 11 2 3

    rv2 = rv.copy()
    rv2['cat_value'] = rv['cat_value'].astype(float)
    rv2 = rv2.sort_values(by=['cat_value', 'mean'])
    rv = rv.loc[list(rv2.index)]

except ValueError:
    rv = rv.sort_values(by=["cat_value", "mean"])

rv = pd.merge(rv_mouseid,rv,on="cat_value")


data = rv
data['perc'] = 100 * data['count'] / data['count'].sum()

# default settings for a boxplot -
# override if other further down

data['_segment_top'] = data['q99']
data['_bar_top'] = data['q75']
data['_bar_median'] = data['median']
data['_bar_bottom'] = data['q25']
data['_segment_bottom'] = data['q01']


In [182]:
plot = figure(x_range = [])
source = ColumnDataSource(data)
plot.x_range.factors = list(set(data['cat_value']))

vbar=plot.vbar(source=source, x='cat_value', top='_bar_top',
                bottom='_bar_bottom', width=0.85, name="barplot",
                fill_color=config.colors.color6,
                line_color="black"),
seg_v_up=plot.segment(source=source, x0='cat_value', x1='cat_value',
                        y0='_bar_top', y1='_segment_top',
                        line_color='black'),
seg_h_up=plot.rect(source=source, x='cat_value', height=0.001,
                    y='_segment_top', width=0.4, line_color='black'),
seg_v_dw=plot.segment(source=source, x0='cat_value', x1='cat_value',
                        y0='_segment_bottom', y1='_bar_bottom',
                        line_color='black'),
seg_h_dw=plot.rect(source=source, x='cat_value', height=0.001,
                    y='_segment_bottom', width=0.4, line_color='black'),
seg_h_med=plot.rect(source=source, x='cat_value', height=0.001,
                    y='_bar_median', width=0.85, line_width=2,
                    line_color='black'),

# source2 = rv_mouseid.loc[np.where(rv_mouseid["cell.state"] == "Cytokines response 1 (CRM-1)")]

jitter_points = plot.scatter(x=jitter('cat_value', width=0.6, range=plot.x_range), y='mean_mouseid', size=9, alpha=0.4, source=source)

show(plot)

In [186]:
data

,cat_value,mouse.id,count_mouseid,mean_mouseid,count,mean,std,median,q25,q75,q01,q99,min,max,perc,_segment_top,_bar_top,_bar_median,_bar_bottom,_segment_bottom
0,Cytokines response 2 (CRM-2),MG277,159,3.316652,9579,2.884441,1.285414,3.135494,2.079442,3.891820,0.0,4.983607,0.0,5.645447,0.125320,4.983607,3.891820,3.135494,2.079442,0.0
1,Cytokines response 2 (CRM-2),MG1482,89,3.961644,9579,2.884441,1.285414,3.135494,2.079442,3.891820,0.0,4.983607,0.0,5.645447,0.125320,4.983607,3.891820,3.135494,2.079442,0.0
2,Cytokines response 2 (CRM-2),MG1676,319,1.650141,9579,2.884441,1.285414,3.135494,2.079442,3.891820,0.0,4.983607,0.0,5.645447,0.125320,4.983607,3.891820,3.135494,2.079442,0.0
3,Cytokines response 2 (CRM-2),MG1705,310,2.154317,9579,2.884441,1.285414,3.135494,2.079442,3.891820,0.0,4.983607,0.0,5.645447,0.125320,4.983607,3.891820,3.135494,2.079442,0.0
4,Cytokines response 2 (CRM-2),MG1223,159,3.747583,9579,2.884441,1.285414,3.135494,2.079442,3.891820,0.0,4.983607,0.0,5.645447,0.125320,4.983607,3.891820,3.135494,2.079442,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,Disease associated (DAM),MG462,55,4.337721,10715,3.602357,1.060216,3.806662,3.091042,4.330733,0.0,5.318120,0.0,5.814131,0.140182,5.318120,4.330733,3.806662,3.091042,0.0
601,Disease associated (DAM),MG467,115,4.575342,10715,3.602357,1.060216,3.806662,3.091042,4.330733,0.0,5.318120,0.0,5.814131,0.140182,5.318120,4.330733,3.806662,3.091042,0.0
602,Disease associated (DAM),MG576,74,3.933738,10715,3.602357,1.060216,3.806662,3.091042,4.330733,0.0,5.318120,0.0,5.814131,0.140182,5.318120,4.330733,3.806662,3.091042,0.0
603,Disease associated (DAM),MG165,710,3.610316,10715,3.602357,1.060216,3.806662,3.091042,4.330733,0.0,5.318120,0.0,5.814131,0.140182,5.318120,4.330733,3.806662,3.091042,0.0


In [64]:
data = pd.DataFrame(dict(
        gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
        gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))
data

NameError: name 'dataset_id' is not defined

In [ ]:
unique_obs = expset.get_obsfields(dataset_id)

In [ ]:
expset.get_dataset(dsid)

In [91]:
datadir = util.get_datadir("h5ad")
dsid = "h.s423a.1"
#print(expset.get_varfields(dsid))
rv = pl.read_parquet(datadir / f"{dsid}.var.prq")



In [206]:
p = figure()
dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id,'umap_1',raw=True)[:,0]
,
        gene2 = expset.get_meta(dataset_id,"umap_2",raw=True)[:,0]
,
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))
        
bins = hexbin(data["gene1"], data["gene2"], 0.1)
source_bin = ColumnDataSource(data=dict(r=[], q=[], counts=[]))
source_bin.data = dict(r=bins.r, q=bins.q, counts=bins.counts)

v_func_alpha  = """
var new_xs = new Array(xs.length)
for(var i = 0; i < xs.length; i++) {
    new_xs[i] = alpha_map[xs[i]]
}
return new_xs
"""
length = len(sorted(source_bin.data["counts"].unique().tolist()))
alpha_map = dict(zip(sorted(source_bin.data["counts"].unique().tolist()),[x/length for x in range(0,length)]))
numerical_alpha_transformer = CustomJSTransform(args={"alpha_map": alpha_map}, v_func=v_func_alpha)


#plot:
t = p.hex_tile(q="q",r="r",size=0.1,source=source_bin,
alpha = transform("counts",numerical_alpha_transformer),line_color = None, color = "black")
# p.scatter(x = "gene1",y="gene2", source = ColumnDataSource(data),size = 0.5,alpha=0.1)
show(p)


In [189]:
from bokeh.util.hex import cartesian_to_axial

###get Data####
# data = pd.DataFrame(dict(
#         gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
#         gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
#         obs = expset.get_meta(dataset_id, "cell.state")[:,0],
#         gene3 = expset.get_gene(dataset_id, "APOC1")[:,0],
#         ))

dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))


#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], 0.1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))
groups = df.groupby(["q","r"])
dicts = []
###assign axial coordinates to get average gene expression###
for key,val in groups.groups.items():
    q,r = key
    counts = len(val)
    avg_exp = data['gene3'].loc[groups.groups.get((q,r))].mean()
    dicts = dicts + [{"q" : q,"r" : r,"counts" : counts, "avg_exp" :avg_exp,"index_list":val}]
    

##use in source
final_result = pd.DataFrame(dicts)
source_bin = ColumnDataSource(final_result)



###mappers###
mapper = LinearColorMapper(
    palette='Magma256',
    low=final_result["avg_exp"].max(),
    high=final_result["avg_exp"].min())

v_func_alpha  = """
var new_xs = new Array(xs.length)
for(var i = 0; i < xs.length; i++) {
    new_xs[i] = alpha_map[xs[i]]
}
return new_xs
"""

###plot###
#make alpha optional => checkbox
#plot any continuous to any continuous
#subset for categorical fields => drop down (mutliselectfield)
#color of the third continious variable is based on the full data with the obs
#first layer is everything (with no colors one color only.) grey light
#second layer is the conitnuous variable colored + the possibility to subset
#have the option to either color or not color 
#widget for hex size
length = len(sorted(np.unique(source_bin.data["counts"]).tolist()))
alpha_map = dict(zip(sorted(np.unique(source_bin.data["counts"]).tolist()),[x/length for x in range(50,length)]))

numerical_alpha_transformer = CustomJSTransform(args={"alpha_map": alpha_map}, v_func=v_func_alpha)

# plot = figure()
# plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,
# alpha = transform("counts",numerical_alpha_transformer),line_color = None, 
# color  = {'field': 'avg_exp', 'transform': mapper})


plot = figure()
plot.hex_tile(q="q",r="r",size=0.1,source=source_bin, 
color  = "#D3D3D3")

plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,
alpha = transform("counts",numerical_alpha_transformer),line_color = None, 
color   = {'field': 'avg_exp', 'transform': mapper})

plot.xaxis.axis_label = "APOE"
plot.yaxis.axis_label = "TREM2"
color_bar1 = ColorBar(color_mapper=mapper, location=(0,0),major_label_text_font_size = "0px",major_tick_in = 2,title = "APOC1 average expression")
plot.add_layout(color_bar1,"right")
plot.extra_y_ranges = {"y2": Range1d(start = final_result["counts"].min(), end = final_result["counts"].max())}
extra_axis = LinearAxis(y_range_name = "y2",axis_label = "Counts by Transparency per Hex tile")
plot.add_layout(extra_axis, 'right')


show(plot)


In [193]:

###get Data####
# data = pd.DataFrame(dict(
#         gene1 = expset.get_gene(dataset_id, "APOE")[:,0],
#         gene2 = expset.get_gene(dataset_id, "TREM2")[:,0],
#         obs = expset.get_meta(dataset_id, "cell.state")[:,0],
#         gene3 = expset.get_gene(dataset_id, "APOC1")[:,0],
#         ))

dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], 0.1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))
groups = df.groupby(["q","r"])
dicts = []
###assign axial coordinates to get average gene expression###
for key,val in groups.groups.items():
    q,r = key
    counts = len(val)
    avg_exp = data['gene3'].loc[groups.groups.get((q,r))].mean()
    dicts = dicts + [{"q" : q,"r" : r,"counts" : counts, "avg_exp" :avg_exp,"index_list":val}]
    

##use in source
final_result = pd.DataFrame(dicts)
source_bin = ColumnDataSource(final_result)



###mappers###
mapper = LinearColorMapper(
    palette='Magma256',
    low=final_result["avg_exp"].max(),
    high=final_result["avg_exp"].min())

# v_func_alpha  = """
# var new_xs = new Array(xs.length)
# for(var i = 0; i < xs.length; i++) {
#     new_xs[i] = alpha_map[xs[i]]
# }
# return new_xs
# """

plot = figure()
# plot.hex_tile(q="q",r="r",size=0.1,source=source_bin, 
# color  = "#D3D3D3")

plot.hex_tile(q="q",r="r",size=0.1,source=source_bin,line_color = None, 
color   = {'field': 'avg_exp', 'transform': mapper},alpha = 1)



show(plot)


In [229]:
from sklearn import preprocessing
gene1 =  preprocessing.normalize(data[["gene1","gene2"]])
gene1

100831

In [241]:
dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
max1 = max(data["gene2"]) - max(data["gene1"])
min1 = min(data["gene2"]) - min(data["gene1"])
q, r = cartesian_to_axial(data["gene1"], data["gene2"], len(data["gene2"])/max1, "pointytop")
df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))

In [242]:
plot.hex_tile(q="q",r="r",size=len(data["gene2"])/max1,source=df,line_color = None, 
color   = {'field': 'avg_exp', 'transform': mapper},alpha = 1)
show(plot)


In [245]:
def _round_hex(q, r ):
    ''' Round floating point axial hex coordinates to integer *(q,r)*
    coordinates.

    This code was adapted from:

        https://www.redblobgames.com/grids/hexagons/#rounding

    Args:
        q (array[float]) :
            NumPy array of Floating point axial *q* coordinates to round

        r (array[float]) :
            NumPy array of Floating point axial *q* coordinates to round

    Returns:
        (array[int], array[int])

    '''
    x = q
    z = r
    y = -x-z

    rx = np.round(x)
    ry = np.round(y)
    rz = np.round(z)

    dx = np.abs(rx - x)
    dy = np.abs(ry - y)
    dz = np.abs(rz - z)

    cond = (dx > dy) & (dx > dz)
    q = np.where(cond              , -(ry + rz), rx)
    r = np.where(~cond & ~(dy > dz), -(rx + ry), rz)

    return q.astype(int), r.astype(int)


def custom_cartesian_to_axial(x, y, sizex: float,  sizey: float, orientation: str, aspect_scale: float = 1):
    ''' Map Cartesion *(x,y)* points to axial *(q,r)* coordinates of enclosing
    tiles.

    This function was adapted from:

        https://www.redblobgames.com/grids/hexagons/#pixel-to-hex

    Args:
        x (array[float]) :
            A NumPy array of x-coordinates to convert

        y (array[float]) :
            A NumPy array of y-coordinates to convert

        size (float) :
            The size of the hexagonal tiling.

            The size is defined as the distance from the center of a hexagon
            to the top corner for "pointytop" orientation, or from the center
            to a side corner for "flattop" orientation.

        orientation (str) :
            Whether the hex tile orientation should be "pointytop" or
            "flattop".

        aspect_scale (float, optional) :
            Scale the hexagons in the "cross" dimension.

            For "pointytop" orientations, hexagons are scaled in the horizontal
            direction. For "flattop", they are scaled in vertical direction.

            When working with a plot with ``aspect_scale != 1``, it may be
            useful to set this value to match the plot.

    Returns:
        (array[int], array[int])

    '''
    HEX_FLAT = [2.0/3.0, 0.0, -1.0/3.0, np.sqrt(3.0)/3.0]
    HEX_POINTY = [np.sqrt(3.0)/3.0, -1.0/3.0, 0.0, 2.0/3.0]

    coords = HEX_FLAT if orientation == 'flattop' else HEX_POINTY

    x =  x / sizex * (aspect_scale if orientation == "pointytop" else 1)
    y = -y / sizey / (aspect_scale if orientation == "flattop" else 1)

    q = coords[0] * x + coords[1] * y
    r = coords[2] * x + coords[3] * y

    return _round_hex(q, r)




In [7]:

dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], size = 1, orientation =  "flattop",aspect_scale = 15)

df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))

plot = figure(match_aspect = True)
plot.hex_tile(q="q",r="r",source=df,line_color = None,alpha = 1)
show(plot)


NameError: name 'cartesian_to_axial' is not defined

In [274]:
dataset_id = "h.man2m.1.sct"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        # gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
q, r = cartesian_to_axial(data["gene1"], data["gene2"], size = 0.1, orientation =  "pointytop")

df = pd.DataFrame(dict(r=r,q=q,avg_exp = None))

plot = figure(match_aspect = True)
plot.hex_tile(q="q",r="r",source=df,line_color = None,alpha = 1)
show(plot)

In [406]:

dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        gene2 = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        ))

#manipulate data to get axial coordinates
max1 = data["gene2"].max() if data["gene2"].max()  > data["gene1"].max()  else data["gene1"].max() 
max2 = data["gene2"].max() if data["gene2"].max()  < data["gene1"].max()  else data["gene1"].max() 
aspect_scale = max1/max2
q, r = cartesian_to_axial(data["gene1"], data["gene2"], size = 30, orientation =  "pointytop", aspect_scale = aspect_scale)

Looking at scaling q,r to original data, after scaling original data to [0,1]

In [14]:

dataset_id = "h.man2m.1.raw"
data = pd.DataFrame(dict(
        gene1 = expset.get_meta(dataset_id, "umap_1", raw=True)[:, 0],
        # gene2 = expset.get_meta(dataset_id, "umap_2", raw=True)[:, 0],
        metadata = expset.get_gene(dataset_id, "MALAT1")[:,0],

        gene3 = expset.get_gene(dataset_id, "APOE")[:,0],
        obs = expset.get_meta(dataset_id, "cell.state")[:,0],
        )) 

#manipulate data to get axial coordinates
# max1 = data["gene2"].max() if data["gene2"].max()  > data["gene1"].max()  else data["gene1"].max() 
# max2 = data["gene2"].max() if data["gene2"].max()  < data["gene1"].max()  else data["gene1"].max() 
# aspect_scale = max1/max2
x = data["gene1"]
y = data["metadata"]
q, r = cartesian_to_axial(x, y, size = 30, orientation =  "pointytop")


In [13]:
# data["gene3"] = data["gene3"]/50
# data["gene3"]

0         0.50
1         0.38
2         0.72
3         0.48
4         0.46
          ... 
100826    0.08
100827    0.04
100828    0.26
100829    0.34
100830    0.08
Name: gene3, Length: 100831, dtype: float64

In [22]:
str(data["gene3"]).encode()

b'0         25\n1         19\n2         36\n3         24\n4         23\n          ..\n100826     4\n100827     2\n100828    13\n100829    17\n100830     4\nName: gene3, Length: 100831, dtype: int64'

In [24]:
str(5.75).encode()

b'5.75'